In [1]:
from kan import KAN
from collections import Counter
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
import torch
from sklearn.utils import shuffle
import random
import numpy as np
torch.set_default_dtype(torch.float32)

In [2]:
import pandas as pd
df = pd.read_csv('/tmp/pycharm_project_763/data/trainCPP.csv')

# 提取序列和标签
sequences = df['sequence'].tolist()
y = df['label'].tolist()

df2= pd.read_csv('/tmp/pycharm_project_763/data/testCPP.csv')

# 提取序列和标签
sequences_test = df2['sequence'].tolist()
y_test = df2['label'].tolist()

In [3]:
def calculate_aac(protein_list):
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

    # 初始化存储结果的列表
    aac_list = []

    for protein_sequence in protein_list:
        # 初始化AAC特征字典
        aac = {aa: 0 for aa in amino_acids}

        # 计算每种氨基酸在序列中的频率
        for aa in protein_sequence:
            if aa in aac:
                aac[aa] += 1

        # 将频率转换为比例
        sequence_length = len(protein_sequence)
        aac = [count / sequence_length for aa, count in aac.items()]

        # 将结果添加到列表中
        aac_list.append(aac)

    return aac_list

In [4]:
aac_train = calculate_aac(sequences)
aac_test = calculate_aac(sequences_test)

In [5]:
import scipy.stats.stats as st
def AAcal(seqcont):
    v=[]
    for i in range(len(seqcont)):
        vtar=seqcont[i]
        vtarv=[]
        vtar7=0
        vtar8=0
        vtar9=0
        s = pd.Series(vtar)
        vtar3=np.mean(vtar)  # These 4 dimensions are relevant statistical terms
        vtar4=st.kurtosis(vtar)
        vtar5=np.var(vtar)
        vtar6=st.skew(vtar)
        #for p in range(len(vtar)): # These 3 dimensions are inspired by PAFIG algorithm
            #vtar7=vtar[p]**2+vtar7
            #if vtar[p]>va:
                #vtar8=vtar[p]**2+vtar8
            #else:
                #vtar9=vtar[p]**2+vtar9
        vcf1=[]
        vcf2=[]
        for j in range(len(vtar)-1): #Sequence-order-correlation terms
            vcf1.append((vtar[j]-vtar[j+1]))
        for k in range(len(vtar)-2):
            vcf2.append((vtar[k]-vtar[k+2]))
        vtar10=np.mean(vcf1)
        vtar11=np.var(vcf1)
        vtar11A=st.kurtosis(vcf1)
        vtar11B=st.skew(vcf1)
        vtar12=np.mean(vcf2)
        vtar13=np.var(vcf2)
        vtar13A=st.kurtosis(vcf2)
        vtar13B=st.skew(vcf2)
        vtarv.append(vtar3)
        vtarv.append(vtar4)
        vtarv.append(vtar5)
        vtarv.append(vtar6)
        #vtarv.append(vtar7/len(vtar))
        #vtarv.append(vtar8/len(vtar))
        #vtarv.append(vtar9/len(vtar))
        vtarv.append(vtar10)
        vtarv.append(vtar11)
        vtarv.append(vtar11A)
        vtarv.append(vtar11B)
        vtarv.append(vtar12)
        vtarv.append(vtar13)
        vtarv.append(vtar13A)
        vtarv.append(vtar13B)
        v.append(vtarv)
    return v

In [25]:
AAC_2_train=AAcal(aac_train)
AAC_2_test=AAcal(aac_test)

In [28]:
AAC_2_train=np.concatenate((AAC_2_train,aac_train),axis=1)
AAC_2_test=np.concatenate((AAC_2_test,aac_test),axis=1)

In [29]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(AAC_2_train, y, random_state=42)
X_test, y_test = shuffle(AAC_2_test, y_test, random_state=42)

In [30]:

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)


In [31]:
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train).view(-1, 1)
X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test).view(-1, 1)


In [32]:
X_train = torch.tensor(X_train ,dtype=torch.float32)
y_train = torch.tensor(y_train,dtype=torch.float32)
X_test = torch.tensor(X_test,dtype=torch.float32)
y_test = torch.tensor(y_test,dtype=torch.float32)

/home/wangli/anaconda3/envs/transformer-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/home/wangli/anaconda3/envs/transformer-env/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/wangli/anaconda3/envs/transformer-env/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing 

In [33]:
X_test[0],X_test.shape

(tensor([ 5.0000e-02,  3.7971e+00,  1.1963e-02,  2.2941e+00, -4.7847e-03,
          1.2591e-02,  4.9100e+00,  1.1076e+00, -5.0505e-03,  2.7982e-02,
          1.1770e+00,  4.0271e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  9.0909e-02,  0.0000e+00,  0.0000e+00,  9.0909e-02,
          3.6364e-01,  3.6364e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  9.0909e-02]),
 torch.Size([299, 32]))

In [34]:
y_test[0],y_test.shape

(tensor([0.]), torch.Size([299, 1]))

In [35]:
dataset = {}
dataset = {
    'train_input': X_train,
    'test_input': X_test,
    'train_label': y_train,
    'test_label': y_test
}

In [38]:
model = KAN(width=[32,3,1], grid=3, k=2, seed=42)

checkpoint directory created: ./model
saving model version 0.0


In [39]:
def train_acc():
    # pred=model(X_train)
    # with torch.no_grad():
    #     pred_labels = (pred > 0.5).float() 
    #     auc = roc_auc_score(y_train.cpu(), pred.cpu())
    # 
    # # 混淆矩阵计算
    #     tn, fp, fn, tp = confusion_matrix(y_train.cpu(), pred_labels.cpu()).ravel()
    # 
    #     # Specificity (SP)
    #     sp = tn / (tn + fp) if (tn + fp) > 0 else 0
    # 
    #     # Sensitivity (SN)
    #     sn = tp / (tp + fn) if (tp + fn) > 0 else 0
    # 
    #     # Matthews Correlation Coefficient (MCC)
    #     mcc = matthews_corrcoef(y_train.cpu(), pred_labels.cpu())
    # print(f"AUC: {auc:.4f}, SP: {sp:.4f}, MCC: {mcc:.4f}, SN: {sn:.4f}")
    return torch.mean((torch.round(model(X_train)[:, 0]) == y_train[:, 0]).float())

def test_acc():
    return torch.mean((torch.round(model(X_test)[:, 0]) == y_test[:, 0]).float())

# results = model.train(dataset, opt="LBFGS", steps=10, metrics=(train_acc, test_acc)) ,lamb=0.001 lamb_entropy=4.,lamb=0.1,lamb_l1=2.5,
# lamb=0.005 train/fit
results = model.fit(dataset, opt="LBFGS", steps=10, metrics=(train_acc, test_acc));
print(results['train_acc'][-1], results['test_acc'][-1])

| train_loss: 2.50e-01 | test_loss: 6.55e-01 | reg: 3.54e+01 | : 100%|█| 10/10 [02:34<00:00, 15.41s/

saving model version 0.1
0.9441580772399902 0.5384615659713745
